In [9]:
from azureml.core import Workspace, Dataset
import urllib.request
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.webservice import LocalWebservice

subscription_id = '04b1ca0e-558c-48d0-ba28-fee796a82076'
resource_group = 'analytics1e5d2caf'
workspace_name = 'UpstreetDevML'

workspace = Workspace(subscription_id, resource_group, workspace_name)

In [23]:
## REGISTER AND CONFIGURE MODEL

model = Model.register(workspace, model_name="test2", model_path="./upstreet.py")

env = Environment.from_pip_requirements(name="project_environment", file_path = "./requirements.txt")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory="./",
    entry_script="./upstreet.py",
)

deployment_config = LocalWebservice.deploy_configuration(port=6799)


Registering model test2


In [24]:
## DEPLOY

service = Model.deploy(
    workspace,
    "myservice",
    [model],
    dummy_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry mlresgistrye5586bbf.azurecr.io
Logging into Docker registry mlresgistrye5586bbf.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM mlresgistrye5586bbf.azurecr.io/azureml/azureml_3953a899d851c6104cb2ca06b98ab928
 ---> 625b4198cdda
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 58a7f3ea67bc
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjA0YjFjYTBlLTU1OGMtNDhkMC1iYTI4LWZlZTc5NmE4MjA3NiIsInJlc291cmNlR3JvdXBOYW1lIjoiYW5hbHl0aWNzMWU1ZDJjYWYiLCJhY2NvdW50TmFtZSI6InVwc3RyZWV0ZGV2bWwiLCJ3b3Jrc3BhY2VJZCI6ImFmM2ZiMTdjLTE3MmMtNDUxYi04YWUyLTQxM2MzOTFlYmYyMSJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in e64f50fff0f9
 ---> 04ca2973087f
Step 4/5 : RUN mv '/var/azureml-app/tmpn2xf_f86.py' /var/azureml-app/main.py
 ---> Running in 8e446ea02c9a
 ---> 8d65d49385c6
Step 5/5 :

In [27]:
## TRY IT OUT

import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6799")
headers = {"Content-Type": "application/json"}
user_properties = {
  "version": "hero-banner-0-3",
  "payload": {
    "arguments": {
        "mostPurchases": "BWS",
        "lastPurchased": "AVIS",
        "groupMostPurchases": "GoodGuys"
    },
  "parameterBounds": {}
  }
} 
user_properties = json.dumps(user_properties)
response = requests.post(uri, data=user_properties, headers=headers)
print(response)
print(response.json())

<Response [200]>
{'productCommonId': 'AVIS'}


In [22]:
service.get_logs()

'2021-07-05T03:41:04,312352139+00:00 - rsyslog/run \n2021-07-05T03:41:04,313351485+00:00 - gunicorn/run \n2021-07-05T03:41:04,315735596+00:00 - iot-server/run \n/usr/sbin/nginx: /azureml-envs/azureml_3c4dd51edc4c5fe2b2bee716e0c6324b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_3c4dd51edc4c5fe2b2bee716e0c6324b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_3c4dd51edc4c5fe2b2bee716e0c6324b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_3c4dd51edc4c5fe2b2bee716e0c6324b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_3c4dd51edc4c5fe2b2bee716e0c6324b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-07-05T03:41:04,318044904+00:00 - nginx/run 

In [28]:
## DEPLOY TO CLOUD
model_version = 'hero-banner-0-3'

tags = {
    'source': 'tutorial',
    'production': False,
    'version': model_version
}

from azureml.core.webservice import AciWebservice
aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores = 0.5, 
    memory_gb = 1, 
    tags=tags, 
    auth_enabled=True)

service = Model.deploy(workspace, 
    "upstreet-hero-banner-0-3", 
    [model], 
    dummy_inference_config, 
    aci_deployment_config)

service.wait_for_deployment(show_output=True)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-07 05:03:46+00:00 Creating Container Registry if not exists.
2021-07-07 05:03:46+00:00 Registering the environment.
2021-07-07 05:03:47+00:00 Use the existing image.
2021-07-07 05:03:48+00:00 Generating deployment configuration.
2021-07-07 05:03:49+00:00 Submitting deployment to compute..
2021-07-07 05:03:54+00:00 Checking the status of deployment upstreet-hero-banner-0-3.